In [2]:
#!/usr/bin/env python3
"""
Find (approximate) ground state energy of a small molecule using QAOA.

Example:
    python ground_state_qaoa.py

This example uses:
 - Qiskit Nature to build the molecular Hamiltonian (PySCF driver)
 - Jordan-Wigner mapping to get a qubit Hamiltonian
 - QAOA as a MinimumEigensolver to find the minimum eigenvalue of the qubit Hamiltonian
"""

import numpy as np

from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp

# Qiskit Nature imports
from qiskit_nature.drivers import PySCFDriver, UnitsType
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

# For reproducibility
SEED = 42
algorithm_globals.random_seed = SEED


def build_qubit_hamiltonian(bond_distance=0.735, basis='sto3g'):
    """
    Build molecular Hamiltonian (qubit operator) for H2 with given bond distance.
    Returns:
      - qubit_op: PauliSumOp representing the qubit Hamiltonian
      - nuclear_repulsion_energy: float
      - num_particles, num_spin_orbitals: ints (info)
    """
    # Define H2 molecule geometry: two H atoms on z-axis separated by bond_distance
    # You may change this string to another molecule; for larger molecules you may need more resources.
    atom_string = f"H 0 0 0; H 0 0 {bond_distance}"

    # Create PySCF driver
    driver = PySCFDriver(atom=atom_string, unit=UnitsType.ANGSTROM, basis=basis)

    # Run driver to get a driver result (contains nuclear repulsion energy, etc.)
    driver_result = driver.run()

    # Build an ElectronicStructureProblem from the driver
    problem = ElectronicStructureProblem(driver)

    # Get the electronic Hamiltonian (second-quantized operator)
    second_q_ops = problem.second_q_ops()
    # The main electronic energy operator is usually the first entry
    electronic_energy_op = second_q_ops[0]

    # Convert to qubit operator (Jordan-Wigner)
    qubit_converter = QubitConverter(mapper=JordanWignerMapper())
    num_particles = problem.num_particles
    qubit_op = qubit_converter.convert(operator=electronic_energy_op, num_particles=num_particles)

    # Ensure it's a PauliSumOp (QAOA expects an Operator-like object)
    if not isinstance(qubit_op, PauliSumOp):
        try:
            qubit_op = PauliSumOp.from_operator(qubit_op)
        except Exception:
            # If conversion fails, try casting to PauliSumOp via repr or other conversions
            qubit_op = PauliSumOp.from_list(qubit_op.to_list())

    nuclear_repulsion_energy = driver_result.nuclear_repulsion_energy

    return qubit_op, float(nuclear_repulsion_energy), problem.num_particles, problem.num_spin_orbitals


def run_qaoa_minimization(qubit_op, quantum_instance, reps=2, maxiter=200):
    """
    Run QAOA to approximately minimize the expectation value of qubit_op.
    Returns the result object from compute_minimum_eigenvalue.
    """
    # Optimizer choice (classical): COBYLA works reasonably for many variational circuits
    optimizer = COBYLA(maxiter=maxiter)

    qaoa = QAOA(optimizer=optimizer, reps=reps, quantum_instance=quantum_instance)

    # QAOA implements MinimumEigensolver API
    result = qaoa.compute_minimum_eigenvalue(qubit_op)
    return result


def main():
    # Parameters (tweak for experiments)
    bond_distance = 0.735  # Angstrom (typical H2 equilibrium)
    basis = 'sto3g'
    reps = 2
    maxiter = 200
    shots = 4096

    print("Building qubit Hamiltonian for H2...")
    qubit_op, nuclear_repulsion, num_particles, num_spin_orbitals = build_qubit_hamiltonian(
        bond_distance=bond_distance, basis=basis
    )

    print(f"Number of particles: {num_particles}")
    print(f"Number of spin orbitals: {num_spin_orbitals}")
    print(f"Nuclear repulsion energy: {nuclear_repulsion:.12f} Ha")
    print(f"Qubit operator (first 10 terms):\n{qubit_op.primitive.to_list()[:10]}")

    # Setup backend & QuantumInstance
    backend = Aer.get_backend("aer_simulator")
    qi = QuantumInstance(
        backend,
        shots=shots,
        seed_simulator=SEED,
        seed_transpiler=SEED,
        optimization_level=1,
    )

    print("Running QAOA minimization (this may take a while)...")
    result = run_qaoa_minimization(qubit_op, quantum_instance=qi, reps=reps, maxiter=maxiter)

    energy_estimate = np.real(result.eigenvalue)
    total_energy = energy_estimate + nuclear_repulsion

    print("\nQAOA result summary:")
    print(f"  - QAOA (qubit Hamiltonian) energy estimate: {energy_estimate:.12f} Ha")
    print(f"  - Nuclear repulsion energy: {nuclear_repulsion:.12f} Ha")
    print(f"  - Total energy (estimate): {total_energy:.12f} Ha")
    print(f"  - Optimal parameters found (theta): {getattr(result, 'optimal_point', None)}")
    print(f"  - Extra result data keys: {list(result.__dict__.keys())}")

    print("\nNotes:")
    print("- The energy found by QAOA is an approximation and depends on 'reps', optimizer, and seeds.")
    print("- For better accuracy: increase 'reps', try different optimizers (SPSA, SLSQP), or use chemistry-tailored ansätze (e.g., UCC) with VQE.")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'Aer' from 'qiskit' (unknown location)

In [1]:
!pip list |grep qiskit

qiskit                    2.2.2
qiskit-algorithms         0.4.0
qiskit-ibm-runtime        0.39.0
qiskit-nature             0.7.2
qiskit-qasm3-import       0.6.0
